In [2]:
# 페가수스 모델은 트랜스포머를 사용하고 문장 요약에 특화된 사정전학습 모델임.
# PEGASUS 라이브러리 설정 및 사전학습 모델 불러오기.

# 구글 코랩에서 PEGASUS의 자동문장 요약을 실행하기 위해 필요한 모듈을 설치합니다.
# 그리고 사전학습 모델인 google/pegasus-xsum을 불러옵니다.
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-xsum').to(device)
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-xsum')
model

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_nor

In [ ]:
# 토크나이징 및 인코딩
# 영어 문장을 준비해서    리스트 타입변수 inputs에 넣으시오 
import re
english_sentence = """
Gyeongbokgung (Korean: 경복궁; Hanja: 景福宮; lit. Radiant Prosperity Palace), also known as Gyeongbok Palace or Gyeongbokgung Palace, was the main royal palace of the Joseon dynasty. Built in 1395, it is located in northern Seoul, South Korea. The largest of the Five Grand Palaces built by the Joseon dynasty, Gyeongbokgung served as the home of the royal/imperial family and the seat of government.

Gyeongbokgung continued to serve as the main palace of the Joseon dynasty until the premises were destroyed by fire during the Imjin War (1592–1598) and abandoned for two centuries. However, in the 19th century, all of the palace's 7,700 rooms were restored under the leadership of Prince Regent Heungseon during the reign of King Gojong. Some 500 buildings were restored on a site of over 40 hectares.[1][2] The architectural principles of ancient Korea were incorporated into the tradition and appearance of the Joseon royal court.

In the early 20th century, much of the palace was systematically destroyed by Imperial Japan during its occupation of Korea.[3] On January 21, 1963, it was designated as a cultural property.[4] Since the 1990s, the walled palace complex is gradually being restored to its original form. It also houses the National Palace Museum and the National Folk Museum within the premises of the complex.

Overview
Gyeongbokgung, located in north of Gwanghwamun Square, was built three years after the Joseon dynasty was founded and it served as its main palace.[5] With the mountain Bugaksan behind it and the Street of Six Ministries (today's Sejongno) outside Gwanghwamun Gate, the main entrance to the palace, Gyeongbokgung was situated in the heart of the Korean capital city. It was steadily expanded before being reduced to ashes during the Japanese invasion of 1592.

For the next 273 years the palace grounds were left derelict until being rebuilt in 1867 under the leadership of Regent Heungseon Daewongun. The restoration was completed on a grand scale, with 330 buildings crowded together in a labyrinthine configuration. Within the palace walls were the Outer Court (oejeon), offices for the king and state officials, and the Inner Court (naejeon), which included living quarters for the royal family as well as gardens for leisure. Within its extensive precincts were other palaces, large and small, including Junggung (the Queen's residence) and Donggung (the Crown Prince's residence).

Due to its status as the symbol of national sovereignty, Gyeongbokgung was extensively damaged during the Japanese occupation of the early 20th century. In 1911, ownership of land at the palace was transferred to the Japanese governor-general. In 1915, on the pretext of holding an exhibition, more than 90% of the buildings were torn down. Following the exhibition, the Japanese leveled whatever still remained and built their colonial headquarters, the Government-General Building (1916–26), on the site. Only a handful of iconic structures survived, including the Throne Hall and Gyeonghoeru Pavilion.

Restoration efforts have been ongoing since 1990. The Government-General Building was removed in 1996 and Heungnyemun Gate (2001) and Gwanghwamun Gate (2006–2010) were reconstructed in their original locations and forms. Reconstructions of the Inner Court and Crown Prince's residence have also been completed.

The current total area is 415,800 square metres (4,476,000 sq ft).[6]

History
14th–16th centuries

King Taejo
Gyeongbokgung was originally constructed in 1394 by King Taejo, the first king and the founder of the Joseon dynasty, and its name was conceived by an influential government minister named Jeong Do-jeon. Afterwards, the palace was continuously expanded during the reign of King Taejong and King Sejong the Great. It was severely damaged by fire in 1553, and its costly restoration, ordered by King Myeongjong, was completed in the following year.

However, four decades later, Gyeongbokgung was burnt to the ground during the Japanese invasions of Korea of 1592–1598 when Koreans angry at the court of King Seonjo for evacuating Seoul torched the royal residence.[7] The royal court was moved to the palace Changdeokgung. Gyeongbokgung site was left in ruins for the next three centuries.[8]

19th century
In 1867, during the regency of Daewongun, the palace buildings were reconstructed and formed a massive complex with 330 buildings and 5,792 rooms. Standing on 4,657,576 square feet (432,703 square meters) of land, Gyeongbokgung again became an iconic symbol for both the Korean nation and the Korean royal family. In 1894, the Japanese occupied the palace and forced Gojong to establish a pro-Japanese government. In 1895, after the assassination of Empress Myeongseong by Japanese agents, her husband, Emperor Gojong, left the palace. The Imperial Family never returned to Gyeongbokgung.[9]

"""
english_sentence = re.sub(r"[:.]\[[0-9]+\]\([0-9]+\)|.?[([][0-9]+[])]|\n|\r", r"", english_sentence)
input = tokenizer(english_sentence, truncation=True, padding='longest', return_tensors="pt").to(device)

{'input_ids': tensor([[  959, 53315, 29622, 49727,   143, 53046,   151,   110,   105,   206,
          9976,  6186,   151,   110,   105,   206,  8514,   107, 48416, 56332,
          6827,   312,   163,   606,   130,   959, 53315, 29622,  6827,   132,
           959, 53315, 29622, 49727,  6827,   108,   140,   109,   674,  7940,
         13915,   113,   109,  7845,   661, 28151,   107,  9973,   115,   305,
         48437,   108,   126,   117,   746,   115,  4905, 18292,   108,   793,
          4611,   107,   139,  1368,   113,   109,  6418,  2913,  6827,   116,
           836,   141,   109,  7845,   661, 28151,   108,   959, 53315, 29622,
         49727,  1502,   130,   109,   238,   113,   109,  7940,   191, 42364,
          7434,   328,   111,   109,  2236,   113,   657,   107,  1064, 53315,
         29622, 49727,  2059,   112,  1278,   130,   109,   674, 13915,   113,
           109,  7845,   661, 28151,   430,   109,  6751,   195,  6487,   141,
          1316,   333,   109,   125,  

In [ ]:
with torch.no_grad():
    model.eval()
    result = model(**input)
    print(result)

TypeError: 'method' object is not subscriptable